In [1]:
from dataset_loader import DatasetLoader
import matplotlib.pyplot as plt

datasetLoader = DatasetLoader('./Dataset')

ModuleNotFoundError: No module named 'kiwisolver._cext'

In [ ]:
train_ds, val_ds, test_ds = datasetLoader.prepare_dataset()

In [ ]:
train_ds

In [ ]:
len(train_ds)

In [ ]:
for img_batch, label_batch in train_ds.take(1):
    img_batch = img_batch.numpy()
    label_batch = label_batch.numpy()

    batch_size = img_batch.shape[0] 

    fig, axes = plt.subplots(batch_size, 2, figsize=(8, batch_size * 3))
    
    for i in range(batch_size):
        axes[i, 0].imshow(img_batch[i])
        axes[i, 0].axis("off")
        axes[i, 0].set_title("Image")


        axes[i, 1].imshow(label_batch[i].squeeze(), cmap="gray")
        axes[i, 1].axis("off")
        axes[i, 1].set_title("Label")

    plt.tight_layout()
    plt.show()